In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

import os
from pathlib import Path

from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage, Timer

from vivarium_helpers.projects.alzheimers import loading, population
from vivarium_helpers.projects.alzheimers.population import RunType
from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

!date
!whoami
!pwd

Tue Feb 17 17:59:29 PST 2026
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


# Record version of Vivarium Helpers

In [3]:
!conda list | grep vivarium

# packages in environment at /ihme/homes/ndbs/miniconda3/envs/vivarium_csu_alzheimers_artifact:
vivarium                  3.4.12                   pypi_0    pypi
vivarium-build-utils      2.0.4                    pypi_0    pypi
vivarium-cluster-tools    2.1.17                   pypi_0    pypi
vivarium-csu-alzheimers   0.1.dev18+g43e997c.d20250808          pypi_0    pypi
vivarium-dependencies     1.0.1                    pypi_0    pypi
vivarium-gbd-access       4.2.4                    pypi_0    pypi
vivarium-helpers          0.2.0                     dev_0    <develop>
vivarium-inputs           6.0.4                    pypi_0    pypi
vivarium-public-health    4.3.5                    pypi_0    pypi


In [4]:
!pip freeze | grep vivarium

vivarium==3.4.12
vivarium_build_utils==2.0.4
vivarium_cluster_tools==2.1.17
-e git+https://github.com/ihmeuw/vivarium_csu_alzheimers.git@263eb3d9b80b13735a7380bb892430eaba92603a#egg=vivarium_csu_alzheimers
vivarium_dependencies==1.0.1
vivarium_gbd_access==4.2.4
-e git+https://github.com/ihmeuw/vivarium_helpers.git@14726994615ddaac5513f19155799d6debf4ecb7#egg=vivarium_helpers
vivarium_inputs==6.0.4
vivarium_public_health==4.3.5


# Define data directories

In [5]:
# Project directory
project_dir = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/')

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_in_progress".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())

# Output from multistate life table
mslt_output_dir = output_dir.parent / 'results_11_14_2025_mslt'
print(mslt_output_dir.exists())
output_dir

True
True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_in_progress')

In [6]:
# # For testing: Run directory containing model 8.3 results for all
# # locations
# model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# # Results directory for model 8.3, for testing
# results_dirs = project_dir / model_run_subdir / 'results/'

# Artifact for models 10.0 - 12.2
artifact_model_number = '10.0'


In [7]:
# Model 8.4 results (final runs for 10/31/25 intermediate results)
# Each batch run contains all locations, 100 random seeds, and 3 or 4 draws
model_run_dir_8_4 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec')

# Model 8.7 results (updated final run completed on 11/10/2025)
model_run_dir_8_7 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec')

# Model 12.2 results (updated final run completed on 02/14/2026)
model_run_dir_12_2 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2')

# Batch runs are stored in "timestamp" subdirectories of the model run
# directory.
# In model 12.2, the run directories are nested under two more levels of
# subfolders: batch number, then model_spec, then timestamp.

# Select a model and filter for testing
batch_run_dirs = loading.list_batch_run_directories(model_run_dir_12_2)#[:2]

batch_run_dirs

['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch5/model_spec/2026_02_13_17_56_25',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch9/model_spec/2026_02_14_06_54_15',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch11/model_spec/2026_02_14_06_55_56',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch8/model_spec/2026_02_13_21_06_03',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch12/model_spec/2026_02_14_07_00_06',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch1/model_spec/2026_02_13_12_41_14',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch3/model_spec/2026_02_13_17_10_34',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch2/model_spec/2026_02_13_15_02_52',
 '/mnt

# Create an AlzheimersResultsProcessor object to process results

In [8]:
results = AlzheimersResultsProcessor('10.0', RunType.FINAL, batch_run_dirs=batch_run_dirs)

In [9]:
results.initial_simulation_population

2000000

In [10]:
results.locations

['United States of America',
 'Brazil',
 'China',
 'Germany',
 'Israel',
 'Japan',
 'Spain',
 'Sweden',
 'Taiwan (Province of China)',
 'United Kingdom']

In [11]:
results.draws

[35,
 187,
 41,
 123,
 248,
 122,
 183,
 103,
 161,
 50,
 24,
 172,
 235,
 245,
 69,
 204,
 219,
 63,
 199,
 6,
 249,
 7,
 236,
 1,
 211]

# Load population data and calculate model scale

In [12]:
with Timer():
    results.load_population_data(append_aggregates=True)
results.ops.describe(results.model_scale)

Elapsed time: 0:00:12.541747


,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
Brazil,25.0,0.645126,0.042517,0.544775,0.618288,0.636986,0.682593,0.718303
China,25.0,0.075453,0.003924,0.067459,0.072855,0.075736,0.077771,0.085044
Germany,25.0,0.548393,0.025209,0.498396,0.536829,0.546442,0.557773,0.604171
Israel,25.0,26.039578,2.496689,20.549962,24.481746,25.631266,28.245708,30.287894
Japan,25.0,0.354952,0.020849,0.308678,0.339599,0.355165,0.367604,0.399881
Spain,25.0,1.865326,0.083683,1.719458,1.822242,1.856133,1.887104,2.134511
Sweden,25.0,7.785869,0.522496,6.614222,7.413090,7.838525,8.131916,8.716418
Taiwan (Province of China),25.0,5.271920,0.276774,4.659120,5.108985,5.251648,5.484657,5.726114
United Kingdom,25.0,1.229875,0.047735,1.167644,1.185930,1.220243,1.267019,1.318293


In [13]:
results.person_time

,location,sex,event_year,input_draw,value,age_group
0,United States of America,Female,2025,35,1.134189e+07,60_to_64
1,United States of America,Female,2025,187,1.101609e+07,60_to_64
2,United States of America,Female,2025,41,1.099131e+07,60_to_64
3,United States of America,Female,2025,123,1.141446e+07,60_to_64
4,United States of America,Female,2025,248,1.116892e+07,60_to_64
...,...,...,...,...,...,...
911995,Spain,Both,2100,249,1.830983e+07,60_plus
911996,Sweden,Both,2100,249,3.882594e+06,60_plus
911997,Taiwan (Province of China),Both,2100,249,8.897937e+06,60_plus
911998,United Kingdom,Both,2100,249,2.338290e+07,60_plus


# Deaths and averted deaths

In [14]:
with Timer():
    results.load_measure_from_batch_runs('deaths', filters=loading.FINAL_RESULTS_FILTERS['deaths'])

print(len(results.data.deaths), 'rows')
results.data.deaths.head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('entity', '=', 'alzheimers_disease_state')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch5/model_spec/2026_02_13_17_56_25/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,deaths,baseline,Female,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,deaths,baseline,Male,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,deaths,bbbm_testing,Female,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,deaths,bbbm_testing,Male,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,deaths,bbbm_testing_and_treatment,Female,0.0,Japan


In [ ]:
# Check that draws in the data match expected draws from the keyspace metadata
assert set(results.data.deaths.input_draw.unique()) == set(results.draws)

In [16]:
deaths_prepped = results.process_deaths(results.data.deaths)
deaths_prepped

47.885514 MB measure
15.965514 MB minuend
31.925514 MB subtrahend
10.269541 MB minuend re-indexed
20.529541 MB subtrahend re-indexed
20.529601 MB difference
36.485646 MB difference with reset index
37.625971 MB final difference


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location,metric,disease_stage
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,Deaths Associated with AD,baseline,Female,0.0,Japan,Number,alzheimers_disease_state
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,Deaths Associated with AD,baseline,Male,0.0,Japan,Number,alzheimers_disease_state
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,Deaths Associated with AD,bbbm_testing,Female,0.0,Japan,Number,alzheimers_disease_state
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,Deaths Associated with AD,bbbm_testing,Male,0.0,Japan,Number,alzheimers_disease_state
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2025,35,Deaths Associated with AD,bbbm_testing_and_treatment,Female,0.0,Japan,Number,alzheimers_disease_state
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,1,Averted Deaths Associated with AD,bbbm_testing_and_treatment,Male,-94.0,Brazil,Number,alzheimers_disease_state
2849996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,211,Averted Deaths Associated with AD,bbbm_testing,Female,0.0,Brazil,Number,alzheimers_disease_state
2849997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,211,Averted Deaths Associated with AD,bbbm_testing,Male,0.0,Brazil,Number,alzheimers_disease_state
2849998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_state,cause,2100,211,Averted Deaths Associated with AD,bbbm_testing_and_treatment,Female,-430.0,Brazil,Number,alzheimers_disease_state


In [17]:
deaths_output = results.summarize_and_beautify(deaths_prepped, 'entity')
deaths_output

Timer SummarizingTimer started
2026-02-17 18:00:37.137181
2026-02-17 18:00:59.710707
2026-02-17 18:06:08.380433
Elapsed time: 331.2553 seconds


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,AD Dementia,Reference,Deaths Associated with AD,Number,1382.167015,1082.075853,1688.446299
1,2025,Brazil,60_to_64,Male,AD Dementia,Reference,Deaths Associated with AD,Number,705.817029,484.524273,894.535775
2,2025,Brazil,60_to_64,Both,AD Dementia,Reference,Deaths Associated with AD,Number,2087.984045,1601.325271,2424.101257
3,2025,Brazil,60_to_64,Female,AD Dementia,BBBM Testing Only,Deaths Associated with AD,Number,1382.167015,1082.075853,1688.446299
4,2025,Brazil,60_to_64,Male,AD Dementia,BBBM Testing Only,Deaths Associated with AD,Number,705.817029,484.524273,894.535775
...,...,...,...,...,...,...,...,...,...,...,...
364795,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing Only,Averted Deaths Associated with AD,Rate,0.000000,0.000000,0.000000
364796,2100,United States of America,60_plus,Both,AD Dementia,BBBM Testing Only,Averted Deaths Associated with AD,Rate,0.000000,0.000000,0.000000
364797,2100,United States of America,60_plus,Female,AD Dementia,BBBM Testing and Treatment,Averted Deaths Associated with AD,Rate,0.000119,0.000088,0.000154
364798,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing and Treatment,Averted Deaths Associated with AD,Rate,0.000073,0.000050,0.000098


In [18]:
deaths_output.to_csv(output_dir / "deaths.csv", index=False)

# Prevalence

In [19]:
filename = 'person_time_alzheimers_disease_and_other_dementias'
short_name = 'person_time_ad'

with Timer():
    results.load_measure_from_batch_runs(filename, short_name, filters=loading.FINAL_RESULTS_FILTERS[filename])

print(len(results.data[short_name]), 'rows')
results.data[short_name].head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025')]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model12.2/batch5/model_spec/2026_02_13_17_56_25/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model10.0/spain.hdf'

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,person_time,baseline,Female,alzheimers_blood_based_biomarker_state,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,person_time,baseline,Female,alzheimers_mild_cognitive_impairment_state,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,person_time,baseline,Female,alzheimers_disease_state,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,person_time,baseline,Male,alzheimers_blood_based_biomarker_state,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2025,35,person_time,baseline,Male,alzheimers_mild_cognitive_impairment_state,0.0,Japan


In [20]:
prevalence_counts = results.process_prevalence_counts(results.data.person_time_ad)
print_memory_usage(prevalence_counts)
print(len(prevalence_counts), 'rows')
prevalence_counts.tail()

107.74 MB 
5130000 rows


,age_group,artifact_path,entity,entity_type,event_year,input_draw,location,measure,scenario,sex,sub_entity,value,metric,disease_stage
5129995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,249,Brazil,Prevalence,bbbm_testing_and_treatment,Female,alzheimers_mild_cognitive_impairment_state,120449.368925,Number,alzheimers_mild_cognitive_impairment_state
5129996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,249,Brazil,Prevalence,bbbm_testing_and_treatment,Female,alzheimers_disease_state,407019.767283,Number,alzheimers_disease_state
5129997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,249,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_blood_based_biomarker_state,80663.496235,Number,alzheimers_blood_based_biomarker_state
5129998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,249,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_mild_cognitive_impairment_state,76104.651608,Number,alzheimers_mild_cognitive_impairment_state
5129999,95_plus,/mnt/team/simulation_science/pub/models/vivari...,alzheimers_disease_and_other_dementias,cause,2100,249,Brazil,Prevalence,bbbm_testing_and_treatment,Male,alzheimers_disease_state,134095.008898,Number,alzheimers_disease_state


In [21]:
with Timer():
    prevalence_output = results.summarize_and_beautify(prevalence_counts)
print_memory_usage(prevalence_output)
print(len(prevalence_output), 'rows')
prevalence_output.tail()

Timer SummarizingTimer started
2026-02-17 18:08:29.446084


2026-02-17 18:09:06.788215
2026-02-17 18:18:12.449076
Elapsed time: 583.0207 seconds
Elapsed time: 0:09:43.020985
21.01924 MB 
656640 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
656635,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing Only,Prevalence,Rate,0.034755,0.032073,0.038168
656636,2100,United States of America,60_plus,Both,AD Dementia,BBBM Testing Only,Prevalence,Rate,0.049242,0.046315,0.052214
656637,2100,United States of America,60_plus,Female,AD Dementia,BBBM Testing and Treatment,Prevalence,Rate,0.059898,0.055782,0.064133
656638,2100,United States of America,60_plus,Male,AD Dementia,BBBM Testing and Treatment,Prevalence,Rate,0.033883,0.031054,0.037295
656639,2100,United States of America,60_plus,Both,AD Dementia,BBBM Testing and Treatment,Prevalence,Rate,0.047967,0.045067,0.050914


In [22]:
# Save prevalence data
prevalence_output.to_csv(output_dir / 'prevalence.csv', index=False)

In [23]:
!date

Tue Feb 17 18:18:17 PST 2026
